<a href="https://colab.research.google.com/github/sanchit2843/SelfDrivingCar/blob/master/steer_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install googledrivedownloader
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='0B-KJCaaF7elleG1RbzVPZWV4Tlk',
                                    dest_path='/content/data.zip',
                                    unzip=True)

Unzipping...Done.


In [0]:
import pandas as pd
data = pd.read_csv('/content/driving_dataset/data.txt', header = None)

In [0]:
train_list = list()
for i in range(len(data)):
  tex = data.iloc[i][0]
  name,angle = tex.split(' ')
  train_list.append((name,angle))

In [0]:
import random
import math
def data_loader(path,label_list,batch_size):
  while True:
    batch_paths =np.random.choice(a = len(label_list),size = batch_size)
    batch_input = []
    batch_output = [] 
    for index in batch_paths:
        input_paths,angle = label_list[index]
        path1 = os.path.join(path,input_paths)
        image = cv2.imread(path1)
        image = image[-150:]
        image = cv2.resize(image,(200,66))
        image = image/255.0
        
        batch_input += [image]
        batch_output += [float(angle)*math.pi/180]
    # Return a tuple of (input,output) to feed the network
    batch_x = np.array( batch_input )
    batch_y = np.array( batch_output )

    yield( batch_x, batch_y )

In [0]:
train = train_list[:int(len(train_list)*0.8)]
test = train_list[int(len(train_list)*0.8):]

In [0]:
import keras
adam = keras.optimizers.Adam(lr = 1e-4,decay=1e-3)

In [60]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten
model = Sequential()
model.add(Conv2D(24, 5, 5, activation='elu', subsample=(2, 2),input_shape = (66,200,3)))
model.add(Conv2D(36, 5, 5, activation='elu', subsample=(2, 2)))
model.add(Conv2D(48, 5, 5, activation='elu', subsample=(2, 2)))
model.add(Conv2D(64, 3, 3, activation='elu'))
model.add(Conv2D(64, 3, 3, activation='elu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(100, activation='elu'))
model.add(Dense(50, activation='elu'))
model.add(Dense(10, activation='elu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer= adam , metrics = ['mse'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_46 (Conv2D)           (None, 31, 98, 24)        1824      
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 14, 47, 36)        21636     
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 5, 22, 48)         43248     
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 3, 20, 64)         27712     
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 1, 18, 64)         36928     
_________________________________________________________________
dropout_9 (Dropout)          (None, 1, 18, 64)         0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 1152)              0         
__________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="elu", input_shape=(66, 200, ..., strides=(2, 2))`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="elu", strides=(2, 2))`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="elu", strides=(2, 2))`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`
  # This is ad

In [62]:
train_gen = data_loader('/content/driving_dataset',train,64)
model.fit_generator(generator = train_gen,steps_per_epoch = len(train)/64,nb_epoch  =32)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<generator..., steps_per_epoch=709.46875, epochs=32)`
  


Epoch 1/32
710/709 [==============================] - 139s 195ms/step - loss: 0.2181 - mean_squared_error: 0.2181
Epoch 2/32
710/709 [==============================] - 145s 205ms/step - loss: 0.1846 - mean_squared_error: 0.1846
Epoch 3/32
710/709 [==============================] - 196s 276ms/step - loss: 0.1695 - mean_squared_error: 0.1695
Epoch 4/32
710/709 [==============================] - 194s 274ms/step - loss: 0.1359 - mean_squared_error: 0.1359
Epoch 5/32
710/709 [==============================] - 192s 271ms/step - loss: 0.1341 - mean_squared_error: 0.1341
Epoch 6/32
710/709 [==============================] - 192s 270ms/step - loss: 0.1205 - mean_squared_error: 0.1205
Epoch 7/32
710/709 [==============================] - 191s 269ms/step - loss: 0.1144 - mean_squared_error: 0.1144
Epoch 8/32
710/709 [==============================] - 189s 266ms/step - loss: 0.1097 - mean_squared_error: 0.1097
Epoch 9/32
710/709 [==============================] - 188s 265ms/step - loss: 0.1025 - m

In [0]:
test_gen =  data_loader('/content/driving_dataset',test,1)

In [0]:
for (image,angle) in test_gen:
  pred = model